# Preprocesado de Datos
 ## Se importan datos usando las funcionalidades de Numpy y Pandas
 
 ### Trabajo de Fin de Grado - Universidad de Burgos
 #### Jesús García Armario

In [1]:
## Imports
import pandas as pd
import numpy as np

### Dataset Servicio Andaluz de Salud:

In [2]:
# Cargamos un dataframe por especialidad
ruta ='.\Listados'
df_plastica = pd.read_excel(ruta+'\listadoPlastica.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_neurocia = pd.read_excel(ruta+'\listadoNeurocia.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_toracica = pd.read_excel(ruta+'\listadoToracica.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_maxilo = pd.read_excel(ruta+'\listadoMaxilo.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_trauma = pd.read_excel(ruta+'\listadoTrauma.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_otorrino = pd.read_excel(ruta+'\listadoOtorrino.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_general = pd.read_excel(ruta+'\listadoGeneral.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])

In [3]:
# Definimos función que recibe el data frame y lo devuelve preprocesado
def preprocesar(df: pd.DataFrame):
    # Eliminamos columnas vacías
    df = df.droplevel(level=1,axis=0)
    df = df.droplevel(level = 1, axis = 1)
    df = df.dropna(axis=1, how='all')
    del df['QUIRÓFANO']
    del df['APELLIDOS, NOMBRE']
    del df['TURNO']
    del df['INICIO']
    del df['FIN']
    df = df.drop(df.columns[[0]],axis=1)

    # Se renombran las columnas repetidas
    cols = []
    count = 1
    countT = 1
    contC = 1
    for i in df.columns:
        if i=='DURACIÓN':
            cols.append('DURACIÓN'+str(count))
            count += 1
            continue
        if i == 'TIPO':
            cols.append('TIPO'+str(countT))
            countT+=1
            continue
        if i=='CARÁCTER ECONÓMICO':
            cols.append('CARÁCTER ECONÓMICO'+str(contC))
            contC+=1
            continue
        cols.append(i)

    df.columns = cols

    # Eliminamos la columna que sobra
    del df['DURACIÓN1']
    # Renombramos la otra
    df.rename(columns = {"TIPO1": "TIPO", "TIPO2": "TURNO", "DURACIÓN2":"DURACIÓN", "CARÁCTER ECONÓMICO1":"CARÁCTER ECONÓMICO",
    "CARÁCTER ECONÓMICO2":"PONDERACIÓN"},inplace=True)

    # Eliminamos filas vacías
    df = df.dropna(axis=0,how = 'all')
    # Codificamos: Cirugía Menor = 0; Cirugía Mayor = 1
    df['TIPO'] = df['TIPO'].replace(['Menor','Mayor'],[0,1])
    # Codificamos: Mañana = 0; Tarde = 1
    df['TURNO'] = df['TURNO'].replace(['Mañana','Tarde'],[0,1])
    # Codificamos: Actividad Ordinaria = 0; Continuidad Asistencial = 1
    df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(['Actividad Ordinaria','Continuidad Asistencial'],[0,1])
    # Eliminamos texto
    df['TIPO'] = df['TIPO'].replace(r'^\D',np.nan,regex=True)
    df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(r'^\D',np.nan,regex=True)
    df['PONDERACIÓN'] = df['PONDERACIÓN'].replace(r'^\D',np.nan,regex=True)
    df['NUHSA'] = df['NUHSA'].replace(r'^\D',np.nan,regex=True)
    # Eliminamos filas nulas
    df = df.dropna(axis=0,how = 'all')

    # Cambiamos el nombre del atributo NUHSA
    df.rename(columns={"NUHSA":"INTERVENCIÓN"},inplace=True)
    df.reset_index(inplace = True)
    del df["level_0"]
    df.rename(columns={"level_1":"NHC"},inplace = True)
    df = df.groupby('NHC', as_index = False).aggregate({"INTERVENCIÓN":"last","TIPO":"first", "TURNO":"first", 
    "CARÁCTER ECONÓMICO":"last","PONDERACIÓN":"last", "DURACIÓN":"first"})

    # Eliminamos los valores que tienen algún nulo
    df = df.dropna(axis=0,how='any')

    return df

In [4]:
# Preprocesamos los datos
df_plastica = preprocesar(df_plastica)
df_neurocia = preprocesar(df_neurocia)
df_toracica = preprocesar(df_toracica)
df_maxilo = preprocesar(df_maxilo)
df_trauma = preprocesar(df_trauma)
df_otorrino = preprocesar(df_otorrino)
df_general = preprocesar(df_general)
# Añadimos una columna con el valor de las especialidades
especialidad = ['PLASTICA'] * len(df_plastica)
df_plastica['ESPECIALIDAD'] = especialidad
especialidad = ['NEUROCIRUGIA'] * len(df_neurocia)
df_neurocia['ESPECIALIDAD'] = especialidad
especialidad = ['TORACICA'] * len(df_toracica)
df_toracica['ESPECIALIDAD'] = especialidad
especialidad = ['MAXILOFACIAL'] * len(df_maxilo)
df_maxilo['ESPECIALIDAD'] = especialidad
especialidad = ['TRAUMATOLOGIA'] * len(df_trauma)
df_trauma['ESPECIALIDAD'] = especialidad
especialidad = ['OTORRINOLARINGOLOGIA'] * len(df_otorrino)
df_otorrino['ESPECIALIDAD'] = especialidad
especialidad = ['GENERAL'] * len(df_general)
df_general['ESPECIALIDAD'] = especialidad

In [7]:
# Unimos los dataframes en uno solo
df = pd.concat([df_plastica,df_neurocia,df_toracica,df_maxilo,df_trauma, df_otorrino, df_general],ignore_index=True)

EXPORTAMOS A ARCHIVO EXTERNO

In [8]:
# Exportamos a Excel
archivo = "ListadoInterv_Preprocesado.xlsx"
df.to_excel(archivo)